In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
dataset = pd.read_csv("../input/league-of-legends-diamond-ranked-games-10-min/high_diamond_ranked_10min.csv")

In [ ]:
dataset.info()
pd.options.display.max_columns= None
dataset.head(10)

In [ ]:
dataset.shape

In [ ]:
# gameId has no role
dataset.drop('gameId',axis=1, inplace=True)


In [ ]:
dataset.head()

In [ ]:
dataset = dataset.sample(frac=1).reset_index(drop=True)

In [ ]:
Y = dataset['blueWins']

In [ ]:
Y.head()

In [ ]:
# For Factor analysis we can remove
X = dataset.drop('blueWins',axis=1, inplace=False)

In [ ]:
sns.heatmap(X.corr())

In [ ]:
fig , ax = plt.subplots(figsize=(25,25))
ax = sns.heatmap(X.corr(), annot=True ,ax=ax, cmap="YlGnBu",linewidths = .5)
ax.set_title("Correlation Between variables", fontsize=36)
plt.show()

In [ ]:
dataset['blueTotalGold'].value_counts()

In [ ]:
# dataset['blueTotalGold'].hist()
# fig, (ax1, ax2) = plt.subplots(1, 2, constrained_layout=True, sharey=True)
# ax1.plot(dataset['blueTotalGold'].hist())

# fig.suptitle('Different types of oscillations', fontsize=16)

In [ ]:
traning_test_split = 0.7
num_example = X.shape[0]
num_train_example = int(np.floor(num_example * traning_test_split))
num_test_example = int(np.ceil(num_example - num_train_example))
print(num_example)
print(num_train_example)
print(num_test_example)

## Mode

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999,epsilon=1e-07, amsgrad=False, name='Adam')

In [ ]:
inputs = tf.keras.Input(shape=(38,))
x = tf.keras.layers.Dense(16, activation=tf.nn.relu)(inputs)
x = tf.keras.layers.Dense(16, activation=tf.nn.relu)(x)
outputs = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

In [ ]:
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
X_train = X.iloc[0:num_train_example, :]
Y_train = Y.iloc[0:num_train_example]

X_test = X.iloc[num_train_example:num_example, :]
Y_test = Y.iloc[num_train_example:num_example]

In [ ]:
print(X_train.shape)
X_test.head()

In [ ]:
BATCH_SIZE = 32
EPOCHS = 300

In [ ]:
model.fit(x=X_train, y=Y_train, batch_size=BATCH_SIZE,epochs=EPOCHS,verbose=1,validation_split=0.2,shuffle=True)

In [ ]:
loss, accuracy = model.evaluate(X_test,Y_test)